<a href="https://colab.research.google.com/github/thanhlong1997/100-nlp-papers/blob/master/phobert_inside.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!wget https://s3.amazonaws.com/models.huggingface.co/bert/vinai/phobert-base/vocab.txt

--2020-10-13 08:11:37--  https://s3.amazonaws.com/models.huggingface.co/bert/vinai/phobert-base/vocab.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.141.206
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.141.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 895321 (874K) [text/plain]
Saving to: ‘vocab.txt’

vocab.txt           100%[===================>] 874.34K   966KB/s    in 0.9s    

2020-10-13 08:11:38 (966 KB/s) - ‘vocab.txt’ saved [895321/895321]



In [37]:
def get_pairs(word):
    """Return set of symbol pairs in a word.
    Word is represented as tuple of symbols (symbols being variable-length strings).
    """
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char

    pairs = set(pairs)
    return pairs
print(get_pairs('Hôm_nay'))

{('n', 'a'), ('m', '_'), ('ô', 'm'), ('a', 'y'), ('_', 'n'), ('H', 'ô')}


In [38]:
with open('bpe.codes', encoding="utf-8") as merges_handle:
  merges = merges_handle.read().split("\n")[:-1]

merges = [tuple(merge.split()[:-1]) for merge in merges]
bpe_ranks = dict(zip(merges, range(len(merges))))
cache = {}

def bpe(token):
  if token in cache:
    return cache[token]
  word = tuple(token)
  word = tuple(list(word[:-1]) + [word[-1] + "</w>"])
  pairs = get_pairs(word)

  if not pairs:
    return token

  while True:
    bigram = min(pairs, key=lambda pair: bpe_ranks.get(pair, float("inf")))
    if bigram not in bpe_ranks:
      break
    first, second = bigram
    new_word = []
    i = 0
    while i < len(word):
      try:
        j = word.index(first, i)
      except ValueError:
        new_word.extend(word[i:])
        break
      else:
        new_word.extend(word[i:j])
        i = j

      if word[i] == first and i < len(word) - 1 and word[i + 1] == second:
        new_word.append(first + second)
        i += 2
      else:
        new_word.append(word[i])
        i += 1
    new_word = tuple(new_word)
    word = new_word
    if len(word) == 1:
      break
    else:
      pairs = get_pairs(word)
  word = "@@ ".join(word)
  word = word[:-4]
  cache[token] = word
  return word

print(bpe('Viblo'))

Vi@@ b@@ lo


In [39]:
import re
def _tokenize(text):
    """Tokenize a string."""
    split_tokens = []

    words = re.findall(r"\S+\n?", text)

    for token in words:
        split_tokens.extend([t for t in bpe(token).split(" ")])
    return split_tokens

print(_tokenize('Hôm_nay trời nóng quá nên tôi ở nhà viết Viblo!'))

['Hôm_nay', 'trời', 'nóng', 'quá', 'nên', 'tôi', 'ở', 'nhà', 'viết', 'Vi@@', 'blo@@', '!']


In [40]:
encoder = {}
bos_token="<s>"
eos_token="</s>"
sep_token="</s>"
cls_token="<s>"
unk_token="<unk>"
pad_token="<pad>"
mask_token="<mask>"


encoder[bos_token] = 0
encoder[pad_token] = 1
encoder[eos_token] = 2
encoder[unk_token] = 3

def add_from_file(f):
  if isinstance(f, str):
      try:
          with open(f, "r", encoding="utf-8") as fd:
              add_from_file(fd)
      except FileNotFoundError as fnfe:
          raise fnfe
      except UnicodeError:
          raise Exception("Incorrect encoding detected in {}, please " "rebuild the dataset".format(f))
      return

  lines = f.readlines()
  for lineTmp in lines:
      line = lineTmp.strip()
      idx = line.rfind(" ")
      if idx == -1:
          raise ValueError("Incorrect dictionary format, expected '<token> <cnt>'")
      word = line[:idx]
      encoder[word] = len(encoder)

add_from_file('vocab.txt')

In [41]:

def _convert_token_to_id(token):
    """ Converts a token (str) in an id using the vocab. """
    return encoder.get(token, encoder.get(unk_token))
print(_convert_token_to_id('Hôm_nay'))

3791


In [42]:
decoder = {v: k for k, v in encoder.items()}
def _convert_id_to_token(index):
    return decoder.get(index, unk_token)

print(_convert_id_to_token(7))

của


In [43]:
def convert_tokens_to_string(tokens):
    out_string = " ".join(tokens)
    out_string = out_string.replace("@@ ", "").strip()
    out_string = out_string.replace('@@','').strip()
    return out_string
  
print(convert_tokens_to_string(['Hôm_nay', 'trời', 'nóng', 'quá' ,'nên', 'tôi', 'ở', 'nhà', 'viết', 'Vi@@', 'blo@@']))

Hôm_nay trời nóng quá nên tôi ở nhà viết Viblo
